<a href="https://colab.research.google.com/github/zamanmiraz/DSandML-Notebooks/blob/main/RAG/01_simple_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/guyernest/advanced-rag.git
%cd advanced-rag
!pip install --upgrade -r requirements.txt
# Install a compatible version of torchvision to address the nms error
!pip install torchvision==0.18.0
!pip install -q -U google-generativeai

In [ ]:
from rich.console import Console
from rich.style import Style
import pathlib
from rich_theme_manager import Theme, ThemeManager

THEMES = [
    Theme(
        name="dark",
        description="Dark mode theme",
        tags=["dark"],
        styles={
            "repr.own": Style(color="#e87d3e", bold=True),      # Class names
            "repr.tag_name": "dim cyan",                        # Adjust tag names
            "repr.call": "bright_yellow",                       # Function calls and other symbols
            "repr.str": "bright_green",                         # String representation
            "repr.number": "bright_red",                        # Numbers
            "repr.none": "dim white",                           # None
            "repr.attrib_name": Style(color="#e87d3e", bold=True),    # Attribute names
            "repr.attrib_value": "bright_blue",                 # Attribute values
            "default": "bright_white on black"                  # Default text and background
        },
    ),
    Theme(
        name="light",
        description="Light mode theme",
        styles={
            "repr.own": Style(color="#22863a", bold=True),          # Class names
            "repr.tag_name": Style(color="#00bfff", bold=True),     # Adjust tag names
            "repr.call": Style(color="#ffff00", bold=True),         # Function calls and other symbols
            "repr.str": Style(color="#008080", bold=True),          # String representation
            "repr.number": Style(color="#ff6347", bold=True),       # Numbers
            "repr.none": Style(color="#808080", bold=True),         # None
            "repr.attrib_name": Style(color="#ffff00", bold=True),  # Attribute names
            "repr.attrib_value": Style(color="#008080", bold=True), # Attribute values
            "default": Style(color="#000000", bgcolor="#ffffff"),   # Default text and background
        },
    ),
]

theme_dir = pathlib.Path("themes").expanduser()
theme_dir.expanduser().mkdir(parents=True, exist_ok=True)

theme_manager = ThemeManager(theme_dir=theme_dir, themes=THEMES)
theme_manager.list_themes()

dark = theme_manager.get("dark")
theme_manager.preview_theme(dark)

In [ ]:
from rich.console import Console

dark = theme_manager.get("dark")
# Create a console with the dark theme
console = Console(theme=dark)

In [ ]:
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
data = (
    pd
    .read_csv('advanced-rag/data/top_rated_wines.csv')
    .query('variety.notna()')
    .reset_index(drop=True)
    .to_dict(orient='records')
)
data[:2]

In [ ]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

# create the vector database client
qdrant = QdrantClient(":memory:")

# create the embedding encoder
encoder = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Create the collection to store the wine rating data
collection_name = "top_wines"
qdrant.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE
    )
)

In [ ]:
# vectorize!
qdrant.upload_points(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data)
    ]
)

In [ ]:
qdrant.get_collection(collection_name=collection_name)

In [ ]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"
query_vector = encoder.encode(user_prompt).tolist()

In [ ]:
# search time for awesome wines

hits = qdrant.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=3
)

In [ ]:
from rich.console import Console
from rich.text import Text
from rich.table import Table
table = Table(title="Retrieval Results", show_lines=True)

table.add_column("Name", style="#e0e0e0")
table.add_column("Region", style="bright_red")
table.add_column("Variety", style="green")
table.add_column("Rating", style="yellow")
table.add_column("Notes", style="#89ddff")
table.add_column("Score", style="#a6accd")

for hit in hits:
    table.add_row(
        hit.payload["name"],
        hit.payload["region"],
        hit.payload["variety"],
        str(hit.payload["rating"]),
        f'{hit.payload["notes"][:50]}...',
        f"{hit.score:.4f}"
    )

table

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from openai import OpenAI
from rich.panel import Panel

client = OpenAI()

# Task
Set up Gemini API using Colab Secrets Manager.

First, install the Google Generative AI library.

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Next, initialize the Gemini model. You can choose a specific model depending on your needs.

In [ ]:
# Initialize the Gemini API
gemini_model = genai.GenerativeModel('gemini-2.0-flash')

Finally, you can test the API to make sure it's working correctly.

In [ ]:
# Now time to connect to the large language model
import google.generativeai as genai
from rich.panel import Panel
from rich.text import Text

# Assuming gemini_model is already initialized in a previous cell
# gemini_model = genai.GenerativeModel('gemini-1.5-flash')

response = gemini_model.generate_content(f"As a wine specialist, suggest an amazing Malbec wine from Argentina based on the following user prompt: {user_prompt}")

response_text = Text(response.text)
styled_panel = Panel(
    response_text,
    title="Wine Recommendation with Gemini",
    expand=False,
    border_style="bold green",
    padding=(1, 1)
)

console.print(styled_panel)

In [ ]:

# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [ ]:
# Use Gemini for Retrieval Augmented Generation (RAG)

completion = gemini_model.generate_content(
    f"As a wine specialist, suggest an amazing Malbec wine from Argentina based on the following user prompt and the provided search results:\n\nUser Prompt: {user_prompt}\n\nSearch Results: {search_results}"
)

response_text = Text(completion.text)
styled_panel = Panel(
    response_text,
    title="Wine Recommendation with Retrieval Augmented Generation (Gemini)",
    expand=False,
    border_style="bold blue",
    padding=(1, 1)
)

console.print(styled_panel)